<a href="https://colab.research.google.com/github/Method-for-Software-System-Development/Cloud_Computing/blob/develop/gui/FaultRepair2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython import get_ipython
from IPython.display import display
!pip install gradio --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.1/54.1 MB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.1/323.1 kB 15.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 60.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 3.7 MB/s eta 0:00:00


In [2]:
import os, subprocess, sys

REPO_DIR = "/content/Cloud_Computing"
LOGIC_DIR = os.path.join(REPO_DIR, "logic")

# Clone the repo if it's not there yet
if not os.path.isdir(REPO_DIR):
    subprocess.run([
        "git", "clone", "-b", "develop",
        "https://github.com/Method-for-Software-System-Development/Cloud_Computing.git",
        REPO_DIR
    ], check=True)

# Pull latest from develop
subprocess.run(["git", "-C", REPO_DIR, "fetch", "origin"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "checkout", "develop"], check=True)
subprocess.run(["git", "-C", REPO_DIR, "pull"], check=True)

# Install importnb (for notebook importing)
%pip install importnb --quiet

# Add logic folder to sys.path so we can import from it
sys.path.append(LOGIC_DIR)

# Import Fault_controller from logic/Fault_controller.ipynb
from importnb import Notebook
with Notebook():
    import Fault_controller as fc

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.2 MB/s eta 0:00:00
omer_cohen 200
shira_gold 200
daniel_bar 200
roni_benami 200
lior_levi 200
yael_elyashiv 200
noa_katz 200
matan_tal 200
or_peled 200
tom_segal 200
Indoor Sensor Data:
Status: 200 | Uploaded: {'Temperature': 26.4, 'Humidity': 41.0, 'Pressure': 974.1, 'Distance': 210.1}
Status: 200 | Uploaded: {'Temperature': 30.2, 'Humidity': 28.7, 'Pressure': 963.5, 'Distance': 8.9}
Status: 200 | Uploaded: {'Temperature': 36.5, 'Humidity': 91.3, 'Pressure': 919.8, 'Distance': 12.2}
Status: 200 | Uploaded: {'Temperature': 27.1, 'Humidity': 73.0, 'Pressure': 1050.4, 'Distance': 205.0}
Status: 200 | Uploaded: {'Temperature': 25.8, 'Humidity': 45.2, 'Pressure': 960.0, 'Distance': 210.0}
Status: 200 | Uploaded: {'Temperature': 32.7, 'Humidity': 29.5, 'Pressure': 940.2, 'Distance': 7.5}
Status: 200 | Uploaded: {'Temperature': 28.9, 'Humidity': 89.9, 'Pressure': 950.6, 'Distance': 250.0}
Status: 200 | Uploaded: {'Temperature': 30.0,

In [41]:
import gradio as gr
from datetime import datetime


# ------ LOGIC -------
def format_main_fault_html(fault):
    if not fault:
        return """
        <div class="no-fault-wrapper">
            <div class="no-fault-message">No active faults at the moment.</div>
        </div>
        """

    severity_class = fault.get("severity", "")
    title = fault.get("title", "")
    timestamp = fault.get("time", "")
    raw_datetime = fault.get("timestamp", "")

    # Extract date from timestamp
    try:
        dt = datetime.fromisoformat(raw_datetime)
        date_str = dt.strftime("%d-%m-%Y")
    except:
        date_str = ""

    # Split sensor line
    title_words = title.split()
    first_line = " ".join(title_words[:2]) if len(title_words) >= 2 else title
    second_line = " ".join(title_words[2:]) if len(title_words) > 2 else ""

    return f"""
    <div class="main-fault-wrapper {severity_class}">
        <div class="main-fault-header">FAULT DETECTED</div>
        <div class="main-fault-subtime">{date_str}, {timestamp}</div>
        <div class="main-fault-title">{first_line}</div>
        <div class="main-fault-subdesc">{second_line}</div>
    </div>
    """


# ----- Fetch data from Firebase -----
active_faults = fc.fb.get_active_faults()

main_fault = None
fault_rows = ""

if active_faults and isinstance(active_faults, dict):
    for title_key, fault in active_faults.items():
        if fault.get("status") != "Active":
            continue

        # Extract time from title key
        if " (" in title_key:
            time_str = title_key.split(" (")[1].rstrip(")")
            try:
                fault["time"] = datetime.strptime(time_str, "%d-%m-%Y %H:%M:%S").strftime("%H:%M")
            except ValueError:
                fault["time"] = ""
        else:
            fault["time"] = ""

        # Set main fault to the first valid one
        if not main_fault:
            main_fault = fault
            continue

        # Build row
        fault_rows += f"""
        <tr>
          <td>
            <button class="table-button" value="{title_key}" name="fault-button">
              {fault['time']}
            </button>
          </td>
          <td>
            <button class="table-button" value="{title_key}" name="fault-button">
              {fault.get('title', '')}
            </button>
          </td>
          <td>
            <button class="table-button" value="{title_key}" name="fault-button">
              {fault.get('severity', '')}
            </button>
          </td>
          <td>
            <button class="table-button" value="{title_key}" name="fault-button">
              {fault.get('status', '')}
            </button>
          </td>
        </tr>
        """

else:
    fault_rows = "<tr><td colspan='4'>No active faults</td></tr>"

# ------- CSS --------
with gr.Blocks(css=r"""
table.main-table {
  width: 100%;
  border-collapse: collapse;
  font-family: sans-serif;
  margin-top: 16px;
}
table.main-table td, table.main-table th {
  border: 1px solid #ccc;
  padding: 12px;
  text-align: left;
}
tr.main-header-row {
  height: 350px;
}
td.left-cell {
  width: 60%;
  vertical-align: top;
}
td.right-cell {
  width: 40%;
  vertical-align: top;
}
tr.main-section-header {
  background-color: #f0f0f0;
  font-weight: bold;
  text-align: center;
}
div.scrollable-inner {
  max-height: 200px;
  height: 200px;
  overflow-y: auto;
  display: block;
}
table.inner-table {
  padding: 0;
  margin: 0;
  width: 100%;
  border-collapse: collapse;
}
table.inner-table th {
  padding: 8px;
  border: 1px solid #ccc;
}

table.inner-table td {
  padding: 0;
  margin: 0;
  border: 1px solid #ccc;
}

table.inner-table thead {
  background-color: #e8e8e8;
}

/* Row buttons styling */
.table-button {
  display: flex;
  width: 100%;
  height: 100%;
  background-color: white;
  color: black;
  border: none;
  margin: 0;
  padding: 12px;
  text-align: left;
  font: inherit;
  cursor: pointer;
  align-items: center;
  justify-content: flex-start;
}

/* Main fault styles */
.main-fault-wrapper {
  height: 100%;
  box-sizing: border-box;
  display: flex;
  flex-direction: column;
  justify-content: center;
  font-family: sans-serif;
  padding: 24px; /* More generous padding */
  border: 1px solid #ccc;
  border-radius: 8px;
}
.main-fault-wrapper.High {
  background-color: #FF0000; /* Bright red */
}
.main-fault-wrapper.Medium {
  background-color: #FFA500; /* Bright orange */
}
.main-fault-wrapper.Low {
  background-color: #FFFF00; /* Bright yellow */
}

/* Main fault layout sections - Styled like alert card */
.main-fault-header {
  font-weight: 700;
  font-size: 20px;
  letter-spacing: 0.5px;
  color: #ffffff;
  text-transform: uppercase;
  margin-bottom: 6px;
}

.main-fault-subtime {
  font-size: 15px;
  color: #ffffff;
  opacity: 0.85;
  margin-bottom: 16px;
}

.main-fault-title {
  font-size: 20px;
  font-weight: bold;
  color: #ffffff;
  margin-bottom: 6px;
}

.main-fault-subdesc {
  font-size: 15px;
  font-weight: 15px;
  color: #ffffff;
  opacity: 0.95;
}

/* No-fault fallback styling */
.no-fault-wrapper {
  background-color: #f9f9f9;
  border: 1px dashed #ccc;
  padding: 12px;
  border-radius: 8px;
  font-family: sans-serif;
  text-align: center;
}
.no-fault-message {
  font-style: italic;
  color: #777;
}
""") as demo:

# ------ HTML -------
    gr.HTML(f"""
    <table class="main-table">
      <tr class="main-header-row">
        <td class="left-cell">
          {format_main_fault_html(main_fault)}
        </td>
        <td class="right-cell">Status: Online</td>
      </tr>
      <tr class="main-section-header">
        <td colspan="2">Active Faults</td>
      </tr>
      <tr>
        <td colspan="2">
          <div class="scrollable-inner">
            <table class="inner-table">
              <thead>
                <tr>
                  <th>Time</th>
                  <th>Sensor</th>
                  <th>Severity</th>
                  <th>Status</th>
                </tr>
              </thead>
              <tbody>
                {fault_rows}
              </tbody>
            </table>
          </div>
        </td>
      </tr>
    </table>
    """)

demo.launch()


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://43ce0c97fde1c0e465.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
